In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import abc
import tensorflow as tf
from tensorflow import keras
import numpy as np
from numpy.random import exponential
import random

from tf_agents.environments import py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.environments import utils
from tf_agents.specs import array_spec
from tf_agents.environments import wrappers
from tf_agents.environments import suite_gym
from tf_agents.trajectories import time_step as ts

from collections import deque

In [2]:
BOARD_SIZE = 8
MAX_HEALTH = 10

Test code for Snake Environment

In [77]:
class Snake():
    
    # Initialize
    def __init__(self, BOARD_SIZE, MAX_HEALTH):  
        # Set Initial Position
        self.path_X = deque()
        self.path_X.append(np.random.randint(low=2, high=BOARD_SIZE-2))
        self.path_X.append(self.path_X[0])
        self.path_X.append(self.path_X[0])
        self.path_Y = deque()
        self.path_Y.append(np.random.randint(low=2, high=BOARD_SIZE-2))
        self.path_Y.append(self.path_Y[0]+1)
        self.path_Y.append(self.path_Y[0]+2)
        
        # Set Initial Health
        self.max_health = MAX_HEALTH
        self.health = MAX_HEALTH
        
        # Set if snake has just eaten
        self.just_eaten = False
    
    # Return all coordinates
    def get_path_coords(self):
        return self.path_X, self.path_Y
    
    def get_head_coords(self):
        return self.path_X[0], self.path_Y[0]
    
    def get_neck_coords(self):
        return self.path_X[1], self.path_Y[1]
    
    def get_tail_coords(self):
        return self.path_X[-1], self.path_Y[-1]
    
    # Return health
    def reduce_health(self):
        self.health-=1
    
    def get_health(self):
        return self.health
    
    def set_full_health(self):
        self.health = self.max_health
        
    def get_just_eaten(self):
        return self.just_eaten
    
    def set_just_eaten(self, boolean):
        self.just_eaten = boolean
    
    def move(self, action):
        i=0; j=0
        if (action==0):
            i = 1
        elif (action==1):
            j = 1
        elif (action==2):
            i = -1
        elif (action==3):
            j = -1
        else:
            raise ValueError('`action` should be 0 or 1 or 2 or 3.')
        self.path_X.appendleft(self.path_X[0]+i)
        self.path_Y.appendleft(self.path_Y[0]+j)
        if not self.just_eaten:
            self.path_X.pop()
            self.path_Y.pop()

In [78]:
class SnakeEnv(py_environment.PyEnvironment):
    
    def reset_board(self):
        
        # Create Snakes
        self.master_snake = Snake(BOARD_SIZE, MAX_HEALTH)
        
        # Create Board
        self._state = [([0]*BOARD_SIZE) for i in range(BOARD_SIZE)]
        self._episode_ended = False 
        master_x_coords, master_y_coords = self.master_snake.get_path_coords()
        for i, (x, y) in enumerate(zip(master_x_coords, master_y_coords)):
                if i == 0:
                    self._state[y][x] = -1
                else:
                    self._state[y][x] = 1
    
        # Set food timer
        self.food_spawn_arr = np.ceil(exponential(5, size=100))
        self.current_food_spawn = 0 #current index of above array
        self.food_timer = 0
        # Maybe store other snakes healths in an array?

        # Set if snake ate during previous turn
        self.just_eaten = False
    
    def __init__(self):
        self._action_spec = array_spec.BoundedArraySpec(
            shape=(), dtype=np.int32, minimum=0, maximum=3, name='action')
        self._observation_spec = array_spec.BoundedArraySpec(
            shape=(BOARD_SIZE,BOARD_SIZE), dtype=np.int32, minimum=-1, name='observation')
        self.reset_board()
        
    def action_spec(self):
        return self._action_spec

    def observation_spec(self):
        return self._observation_spec

    def _reset(self):
        self.reset_board()
        return ts.restart(np.array(self._state, dtype=np.int32))
            
        
    def foodspawn_assist(self):
        coords = list(zip(np.where(np.array(self._state)==0)[0],
                          np.where(np.array(self._state)==0)[1]))
        coord = random.choice(coords)
        self._state[coord[0]][coord[1]] = 2
        return None

    def _step(self, action):
        
        if self._episode_ended:
            # The last action ended the episode. Ignore the current action and start
            # a new episode.
            return self.reset()
        
        # Reduce Health
        self.master_snake.reduce_health()
        
        # Get Previous tail coordinates
        master_tail_x, master_tail_y = self.master_snake.get_tail_coords()
        
        # Apply Action
        try: self.master_snake.move(action)
        except ValueError as error:
            raise ValueError('`action` should be 0 or 1 or 2 or 3.')
        
        # Get  New Head Coords
        master_head_x, master_head_y = self.master_snake.get_head_coords()
        master_neck_x, master_neck_y = self.master_snake.get_neck_coords()
        
        # Spawn Food
        self.food_timer +=1
        if (self.food_spawn_arr[self.current_food_spawn]==self.food_timer):
            # Spawn Food
            self.foodspawn_assist()
            # Reset Timer
            self.food_timer = 0
            self.current_food_spawn = (self.current_food_spawn+1)%len(self.food_spawn_arr)
        
        # Check to see if out of bounds or hit itself
        if (master_head_x == BOARD_SIZE or master_head_y == BOARD_SIZE \
            or master_head_x == -1 or master_head_y == -1 \
            or self._state[master_head_y][master_head_x] == 1):
            self._episode_ended = True
        
        # Else perform snake step
        else:
            # Check What block snake has landed on. If 2 then
            # it landed on food
            block = self._state[master_head_y][master_head_x]

            # Set Head Value to -1
            self._state[master_head_y][master_head_x] = -1
            # Set Neck to 1
            self._state[master_neck_y][master_neck_x] = 1
            # If snake has not just eaten then set tail location to zero
            if not(self.master_snake.get_just_eaten()):
                self._state[master_tail_y][master_tail_x] = 0
            self.master_snake.set_just_eaten(False)

            # Check if snake consumed food
            if(block==2):
                self.master_snake.set_full_health()
                self.master_snake.set_just_eaten(True)

        # If out of health then die
        if self.master_snake.get_health() == 0:
            return self.reset()
        
        # If Episode Ended
        if self._episode_ended:
            reward = 0
            return ts.termination(np.array(self._state, dtype=np.int32), reward)
        else:
            return ts.transition(
            np.array(self._state, dtype=np.int32), reward=1.0, discount=1.0) 

# Testing

In [79]:
environment = SnakeEnv()
utils.validate_py_environment(environment, episodes=5)

In [80]:
env = SnakeEnv()

In [107]:
env.reset()

TimeStep(step_type=array(0), reward=array(0., dtype=float32), discount=array(1., dtype=float32), observation=array([[ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0, -1,  0,  0,  0],
       [ 0,  0,  0,  0,  1,  0,  0,  0],
       [ 0,  0,  0,  0,  1,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0]]))

In [133]:
env.step(0)

TimeStep(step_type=array(2), reward=array(0., dtype=float32), discount=array(0., dtype=float32), observation=array([[ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  2,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0, -1,  1,  1,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  2,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0]]))

# Later Stuff

In [6]:
env = SnakeEnv()
tf_env = tf_py_environment.TFPyEnvironment(env)

# Train Network

In [15]:
max_episode_steps = 27000 # <=> 108k ALE frames since 1 step = 4 frames

## Training Architecture

For more details see 649-650. For now we need to create
1. A deep Q-Network
2. DQN Agent (takes care of creating the collect policy from replay buffer)
3. Replay buffer (and observer to write to it)
4. Some training metrics
5. The driver
6. The data set

### Deep-Q Network and DQN Agent

Heres the network. It has a preprocessing layer that normalizes all the entries. Next we apply a convolutional neural netork (chap 14). Lastly, it applies a dense layer with 512 inputs, and then automatically a dense layer with 4 outputs (one for each Q value). All these layers use ReLU activation functions (except the last one which has no activation function).

In [372]:
from tf_agents.networks.q_network import QNetwork

preprocessing_layer = keras.layers.Lambda(
                          lambda obs: tf.cast(obs, np.float32) / 2.)
conv_layer_params=[(8, 3, 2), (6, 3, 2)]
fc_layer_params=[512] # number of neurons for each dense layer (only 1 layer here)

q_net = QNetwork(
    tf_env.observation_spec(),
    tf_env.action_spec(),
    preprocessing_layers=preprocessing_layer,
    conv_layer_params=conv_layer_params,
    fc_layer_params=fc_layer_params)

In [373]:
tf_env.observation_spec()

BoundedTensorSpec(shape=(8, 8), dtype=tf.int32, name='observation', minimum=array(-1), maximum=array(2147483647))

Heres the agent. 

In [374]:
from tf_agents.agents.dqn.dqn_agent import DqnAgent

# Create variable that counts the number of training steps
train_step = tf.Variable(0)
# Create optimizer 
optimizer = tf.compat.v1.train.RMSPropOptimizer(learning_rate=2.5e-4, decay=0.95, momentum=0.0,
                                     epsilon=0.00001, centered=True)
# Computes epsilon for epsilon greedy policy given the training step
epsilon_fn = keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=1.0, # initial ε
    decay_steps=10000, 
    end_learning_rate=0.01) # final ε

agent = DqnAgent(tf_env.time_step_spec(),
                 tf_env.action_spec(),
                 q_network=q_net,
                 optimizer=optimizer,
                 target_update_period=2000, # <=> 32,000 ALE frames
                 td_errors_loss_fn=keras.losses.Huber(reduction="none"),
                 gamma=0.99, # discount factor
                 train_step_counter=train_step,
                 epsilon_greedy=lambda: epsilon_fn(train_step))
agent.initialize()

ValueError: Input 0 of layer EncodingNetwork/conv2d is incompatible with the layer: expected ndim=4, found ndim=3. Full shape received: [0, 8, 8]
  In call to configurable 'DqnAgent' (<function DqnAgent.__init__ at 0x000001F3E31990D8>)

### Replay Buffer and Observer

TF-Agents provides some replay buffer implementation in the tf_agents.replay_buffers package.

In [19]:
from tf_agents.replay_buffers import tf_uniform_replay_buffer

replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    # Determines the data spec type
    data_spec=agent.collect_data_spec,
    # The number of trajectories added at each step
    batch_size=tf_env.batch_size,
    # This can store 1 million trajectories (note: requires a lot of RAM)
    max_length=1000000)

# Create the observer that adds trajectories to the replay buffer
replay_buffer_observer = replay_buffer.add_batch

This class increments every time it is called

In [24]:
class ShowProgress:
    def __init__(self, total):
        self.counter = 0
        self.total = total
    def __call__(self, trajectory):
        if not trajectory.is_boundary():
            self.counter += 1
        if self.counter % 100 == 0:
            print("\r{}/{}".format(self.counter, self.total), end="")

### Training Metrics

These training metrics can be stored during training (useful for outputting verbose things during training).

In [20]:
from tf_agents.metrics import tf_metrics

train_metrics = [
    tf_metrics.NumberOfEpisodes(),
    tf_metrics.EnvironmentSteps(),
    tf_metrics.AverageReturnMetric(),
    tf_metrics.AverageEpisodeLengthMetric(),
]

The metrics can be logged as follows:

In [21]:
from tf_agents.eval.metric_utils import log_metrics
import logging
logging.getLogger().setLevel(logging.INFO)
log_metrics(train_metrics)

INFO:absl: 
		 NumberOfEpisodes = 0
		 EnvironmentSteps = 0
		 AverageReturn = 0.0
		 AverageEpisodeLength = 0.0


### Collect Driver

The driver explores the environment using a given policy, collects experiences, and broadcasts them to some observers (and then the observers typically place them in the replay buffer).
1. Driver passes current **time step** to collect policy which then chooses an action and returns an **action step** object containing the action
2. The driver passes the action to the environment, which then returns the next step
3. The driver creates a trajectory object to represent this transition and broadcasts it to the observer

There are two main types of drivers:
1. DynamicStepDriver: Collects experiences for certain number of steps
2. DynamicEpisodeDriver: Collects experiences for certain number of episodes

In our case we want to for four steps for each training iteration (4 steps allows us to see where ball is going). We create it as follows:

In [22]:
from tf_agents.drivers.dynamic_step_driver import DynamicStepDriver

collect_driver = DynamicStepDriver(
    tf_env, # Env to play with
    agent.collect_policy, # Collect policy of the agent
    observers=[replay_buffer_observer] + train_metrics, # pass to all observers
    num_steps=update_period) # collect 4 steps for each training iteration

### Initializing the Replay Buffer with Experiences

In [25]:
from tf_agents.policies.random_tf_policy import RandomTFPolicy

initial_collect_policy = RandomTFPolicy(tf_env.time_step_spec(),
                                        tf_env.action_spec())
init_driver = DynamicStepDriver(
    tf_env,
    initial_collect_policy,
    observers=[replay_buffer.add_batch, ShowProgress(20000)],
    num_steps=20000) # <=> 80,000 ALE frames
final_time_step, final_policy_state = init_driver.run()

20000/20000

### Creating the Dataset

This has to do with sampling batches of trajectories from the replay buffer. To sample them, you just need to call the "get_next()" method. This also returns a BufferInfo object that contains the sample identifiers and their sampling probabilities.
* The following code samples a small batch of two trajectories (subepisodes) each containing three consecutive steps.

In [26]:
tf.random.set_seed(888) # chosen to show an example of trajectory at the end of an episode

trajectories, buffer_info = replay_buffer.get_next(
    sample_batch_size=2, num_steps=3)

Can get all attributes of these trajectories

In [27]:
trajectories._fields

('step_type',
 'observation',
 'action',
 'policy_info',
 'next_step_type',
 'reward',
 'discount')

Shape of observations:

In [28]:
trajectories.observation.shape

TensorShape([2, 3, 84, 84, 4])

Two trajectories, each with three steps, each steps observation is 84x84 x4 (4 overlaid image). Can also convert  to time_steps, action_steps, and next_time_steps

In [29]:
from tf_agents.trajectories.trajectory import to_transition

time_steps, action_steps, next_time_steps = to_transition(trajectories)
time_steps.observation.shape

TensorShape([2, 2, 84, 84, 4])

Now lets convert the dataset to a tensorflow dataset so we can take advantage of parallelization. We sample batches of 64 trajectories, each trajectory with 2 steps. The data set processes three elements in parallel, and prefetches 3 batches during training.

In [34]:
dataset = replay_buffer.as_dataset(
    sample_batch_size=64,
    num_steps=2,
    num_parallel_calls=3).prefetch(3)

### Creating Training Loop

To speed up training we convert main functions to TensorFlow Fucntions. For this we use a wrapper

In [35]:
from tf_agents.utils.common import function

collect_driver.run = function(collect_driver.run)
agent.train = function(agent.train)

Heres a function that runs training for $n$ iterations

In [36]:
def train_agent(n_iterations):
    time_step = None
    # Get initial policy state
    policy_state = agent.collect_policy.get_initial_state(tf_env.batch_size)
    # Create iterator over dataset and loop
    iterator = iter(dataset)
    for iteration in range(n_iterations):
        # Pass current time step and policy state to get next time step and policy state
        # Collects experience for 4 steps then broadcasts trajectories to replay buffer
        time_step, policy_state = collect_driver.run(time_step, policy_state)
        # Sample a batch of trajectories from the dataset, pass to the train method
        trajectories, buffer_info = next(iterator)
        train_loss = agent.train(trajectories)
        print("\r{} loss:{:.5f}".format(
            iteration, train_loss.loss.numpy()), end="")
        if iteration % 1000 == 0:
            log_metrics(train_metrics)

Train the agent!

In [45]:
train_agent(n_iterations=10000)

INFO:absl: 
		 NumberOfEpisodes = 232
		 EnvironmentSteps = 40004
		 AverageReturn = 0.8999999761581421
		 AverageEpisodeLength = 170.39999389648438


997 loss:0.00757

INFO:absl: 
		 NumberOfEpisodes = 254
		 EnvironmentSteps = 44004
		 AverageReturn = 1.399999976158142
		 AverageEpisodeLength = 182.39999389648438


1997 loss:0.00033

INFO:absl: 
		 NumberOfEpisodes = 279
		 EnvironmentSteps = 48004
		 AverageReturn = 0.30000001192092896
		 AverageEpisodeLength = 150.3000030517578


...

8000 loss:0.00015

INFO:absl: 
		 NumberOfEpisodes = 410
		 EnvironmentSteps = 72004
		 AverageReturn = 0.8999999761581421
		 AverageEpisodeLength = 176.60000610351562


8999 loss:0.00011

INFO:absl: 
		 NumberOfEpisodes = 432
		 EnvironmentSteps = 76004
		 AverageReturn = 1.399999976158142
		 AverageEpisodeLength = 187.10000610351562


9999 loss:0.00030